## Spherical Harmonics

In [30]:
import numpy as np
from scipy.special import sph_harm
from scipy.spatial.transform import Rotation
import quaternionic
import spherical

from IPython.display import IFrame
import plotly.graph_objects as go
from plotly.subplots import make_subplots

layout = dict(
    font_family="JuliaMono",
    showlegend=False,
    margin=go.layout.Margin(l=0, r=0, b=0, t=0),
    paper_bgcolor='rgba(0,0,0,0)',
)

In [2]:
# We follow the mathematics convention, theta is the azimuthal angle [0, 2π), phi is the polar angle [0, π]
phi, theta = np.mgrid[0:np.pi:128j, 0:2*np.pi:128j]

r = 1
x = r * np.cos(theta) * np.sin(phi)
y = r * np.sin(theta) * np.sin(phi)
z = r * np.cos(phi)

f = np.sqrt(0.75 / np.pi) * y / r + 0.5 * np.sqrt(15 / np.pi) * x * z / r
f = f / np.linalg.norm(f)

In [3]:
fig = make_subplots(rows=1, cols=1, specs=[[{'is_3d': True}]], subplot_titles=[r'$Y_{lm}$'])

fig.add_trace(
  go.Surface(x=x, y=y, z=z, surfacecolor=f, colorscale='RdBu_r')
)

fig.update_layout(layout)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html('example_signal.html')
IFrame('./example_signal.html', 1000, 500)

In [4]:
fig = make_subplots(rows=1, cols=1, specs=[[{'is_3d': True}]])

fig.add_trace(
  go.Surface(x=x * np.abs(f), y=y * np.abs(f), z=z * np.abs(f), surfacecolor=f, colorscale='RdBu_r')
)

fig.update_layout(layout)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html('example_signal_glyph.html')
IFrame('./example_signal_glyph.html', 1000, 500)

### Spherical Harmonics

Spherical harmonics require 2 indices, the degree $l$, similar to the frequency in the Fourier series, and the order $m$.

$$
f(\theta, \phi) = \sum_{l=0}^{\infty} \sum_{m=-l}^{l} f_{lm} Y_{lm}(\theta, \phi)
$$

In [5]:
Y = sph_harm(-1, 5, theta, phi).real

In [6]:
fig = make_subplots(rows=1, cols=2, shared_xaxes=True, shared_yaxes=True,
                    specs=[[{'is_3d': True}, {'is_3d': True}]])

fig.add_trace(
    go.Surface(x=x, y=y, z=z, surfacecolor=Y, colorscale='RdBu_r'),
    1, 1
)
fig.add_trace(
    go.Surface(x=x * np.abs(Y), y=y * np.abs(Y), z=z * np.abs(Y), 
        surfacecolor=Y, colorscale='RdBu_r'),
    1, 2
)

fig.update_layout(layout)

fig.update_traces(showscale=False)
# fig.show()
fig.write_html('example_sph_harmonics.html')
IFrame('./example_sph_harmonics.html', 1000, 500)

In [7]:
num_degrees = 4
max_order = 2 * num_degrees + 1
Y = [sph_harm(np.arange(-l, l+1)[:, None, None], l, theta[None, ...], phi[None, ...]).real
     for l in np.arange(num_degrees+1)]

In [8]:
fig = make_subplots(rows=num_degrees+1, cols=max_order,
                    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x, y=y, z=z, surfacecolor=Y[l][m+l], colorscale='RdBu_r'),
          l+1, m+num_degrees+1
        )

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
fig.update_traces(showscale=False)

fig.write_image("../img/sph_harmonics.svg")
# fig.write_html("sph_harmonics.html")
# IFrame('./sph_harmonics.html', 1000, 500)

![Spherical Harmonics](./../img/sph_harmonics.svg)

In [9]:
fig = make_subplots(rows=num_degrees+1, cols=max_order,
                    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x * np.abs(Y[l][m+l]), y=y * np.abs(Y[l][m+l]), z=z * np.abs(Y[l][m+l]), surfacecolor=Y[l][m+l], colorscale='RdBu_r'),
          l+1, m+num_degrees+1
        )

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)

fig.write_image("../img/glyph_sph_harmonics.svg")
# fig.write_html("glyph_sph_harmonics.html")
# IFrame('./glyph_sph_harmonics.html', 1000, 500)

![Spherical Harmonics](../img/glyph_sph_harmonics.svg)

In [10]:
theta_linspace = np.linspace(0, 2 * np.pi, 16, endpoint=True)
Y_21_rot = [(sph_harm(1, 2, theta, phi) * np.exp(1j * theta_0)).real
            for theta_0 in theta_linspace]
Y_31_rot = [(sph_harm(1, 3, theta, phi) * np.exp(1j * theta_0)).real
            for theta_0 in theta_linspace]

In [11]:
fig = go.Figure(
    data=go.Surface(x=x * np.abs(Y_31_rot[0]), y=y * np.abs(Y_31_rot[0]), z=z * np.abs(Y_31_rot[0]),
                    surfacecolor=Y_31_rot[0], colorscale='RdBu_r', name='0'),
    frames=[go.Frame(data=go.Surface(x=x * np.abs(Y_31_rot[t]), y=y * np.abs(Y_31_rot[t]), z=z * np.abs(Y_31_rot[t]),
                                     surfacecolor=Y_31_rot[t], colorscale='RdBu_r', name=str(t)))
            for t in range(1, len(theta_linspace))]
)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html("example_rot_glyph_sph_harmonics.html")
IFrame('./example_rot_glyph_sph_harmonics.html', 1000, 500)

In [12]:
num_degrees = 2
max_order = 2 * num_degrees + 1
theta_linspace = np.linspace(0, 2 * np.pi, 32, endpoint=True)

Y_rot = np.empty((len(theta_linspace), num_degrees+1, max_order, theta.shape[0], phi.shape[0]))
for i, theta_0 in enumerate(theta_linspace):
    for l in np.arange(num_degrees+1):
        harmonics = (sph_harm(np.arange(-l, l+1)[:, None, None], l, theta[None, ...], phi[None, ...])
                     * np.exp(1j * np.arange(-l, l+1)[:, None, None] * theta_0)).real
        Y_rot[i, l, num_degrees-l:num_degrees+l+1] = harmonics

In [13]:
frames = [
    dict(
        name=str(t),
        data=[go.Surface(x=x * np.abs(Y_rot[t, l, num_degrees+m]), y=y * np.abs(Y_rot[t, l, num_degrees+m]),
                          z=z * np.abs(Y_rot[t, l, num_degrees+m]), surfacecolor=Y_rot[t, l, num_degrees+m], 
                          colorscale='RdBu_r')
              for l in range(num_degrees+1)
              for m in range(-l, l+1)],
        traces=list(range((num_degrees+1)**2)),
    )
    for t in range(1, len(theta_linspace))
]

In [14]:
fig = make_subplots(
    rows=num_degrees+1, cols=max_order,
    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x * np.abs(Y_rot[0, l, num_degrees+m]), y=y * np.abs(Y_rot[0, l, num_degrees+m]),
                     z=z * np.abs(Y_rot[0, l, num_degrees+m]), surfacecolor=Y_rot[0, l, num_degrees+m],
                     colorscale='RdBu_r'),
          l+1, num_degrees+m+1
        )
fig.update(frames=frames)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)

fig.write_html("full_rot_glyph_sph_harmonics.html")
IFrame('./full_rot_glyph_sph_harmonics.html', 1000, 500)

In [15]:
frames = [
    dict(
        name=str(t),
        data=[go.Surface(x=x, y=y, z=z, surfacecolor=Y_rot[t, l, num_degrees+m], 
                         colorscale='RdBu_r')
              for l in range(num_degrees+1)
              for m in range(-l, l+1)],
        traces=list(range((num_degrees+1)**2)),
    )
    for t in range(1, len(theta_linspace))
]

In [16]:
fig = make_subplots(rows=num_degrees+1, cols=max_order,
                    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x, y=y, z=z, surfacecolor=Y_rot[0, l, num_degrees+m],
                     colorscale='RdBu_r'),
          l+1, num_degrees+m+1
        )
fig.update(frames=frames)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)

fig.write_html("full_rot_sph_harmonics.html")
IFrame('./full_rot_sph_harmonics.html', 1000, 500)

## Dirac Delta Approximation

In [17]:
phi, theta = np.mgrid[0:np.pi:128j, 0:2*np.pi:128j]

r = 1
x = r * np.cos(theta) * np.sin(phi)
y = r * np.sin(theta) * np.sin(phi)
z = r * np.cos(phi)

f = np.zeros_like(x)
f[10, 20] = 1.0

In [18]:
fig = go.Figure(data=go.Cone(x=[0], y=[0], z=[0], u=[x[10, 20]], v=[y[10, 20]], w=[z[10, 20]]))
fig.update_layout(layout)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html('delta_on_sphere.html')
IFrame('./delta_on_sphere.html', 1000, 500)

In [19]:
def num_coeff(l_max):
    return (l_max + 1) ** 2

num_degrees = 10
max_order = 2 * num_degrees + 1
Y = np.concatenate(
    [sph_harm(np.arange(-l, l+1)[:, None, None], l, theta[None, ...], phi[None, ...]).real
     for l in np.arange(num_degrees)], 0)

l_max = [3, 5, 7, 9]
delta_recon = [
    (Y[:num_coeff(l), [[10]], [[20]]] * Y[:num_coeff(l)]).sum(0)
    for l in l_max]

In [20]:
rows, cols = 2, len(l_max)
specs = [[{'is_3d': True} for i in range(cols)]
         for j in range(rows)]
fig = make_subplots(rows=rows, cols=cols, specs=specs)

for col, d in enumerate(delta_recon, start=1):
    fig.add_trace(
        go.Surface(x=x, y=y, z=z, surfacecolor=d, colorscale='RdBu_r'), row=1, col=col)
for col, d in enumerate(delta_recon, start=1):
    fig.add_trace(
        go.Surface(x=x * np.abs(d), y=y * np.abs(d),
                   z=z * np.abs(d), surfacecolor=d, colorscale='RdBu_r'), row=2, col=col)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html('delta_sph_harmonics.html')
IFrame('./delta_sph_harmonics.html', 1000, 500)

# Arbitrary Rotations - Wigner-D Matrix

In [22]:
num_timesteps = 16
R = Rotation.from_rotvec(np.ones((num_timesteps, 3)) * np.linspace(0.0, 2 * np.pi, num_timesteps, endpoint=False)[:, None])
# R = Rotation.from_rotvec(np.concatenate([np.zeros((num_timesteps, 1)), np.ones((num_timesteps, 1)), np.zeros((num_timesteps, 1))],-1) * np.linspace(0.0, 2 * np.pi, num_timesteps, endpoint=False)[:, None])
R = R.as_matrix()

In [23]:
def cum_coeff(l):
    return int(np.round((l + 1) * (2 * l + 1) * (2 * l + 3) / 3))

def block_diag(x):
    y = np.zeros((x[0].shape[0], np.sum([xi.shape[1] for xi in x]),
                  np.sum([xi.shape[2] for xi in x])), dtype=x[0].dtype)
    r = 0
    c = 0
    for xi in x:
        y[:, r:r+xi.shape[1], c:c+xi.shape[2]] = xi
        r += xi.shape[1]
        c += xi.shape[2]
    return y

l_max = 2
wigner = spherical.Wigner(l_max)

D = wigner.D(quaternionic.array.from_rotation_matrix(R))
    
splits = [cum_coeff(x) for x in range(l_max+1)]
D = np.split(D, splits[:-1], axis=-1)
D = [Di.reshape(-1, 2 * i + 1, 2 * i + 1) for i, Di in enumerate(D)]
D = block_diag(D)

In [24]:
num_degrees = 2
max_order = 2 * num_degrees + 1

Y_rot = np.empty((num_timesteps, num_degrees+1, max_order, theta.shape[0], phi.shape[0]))
for i, Di in enumerate(D):
    for l in np.arange(num_degrees+1):
        rot = Di[l**2:(l+1)**2, l**2:(l+1)**2]
        sph = sph_harm(np.arange(-l, l+1)[:, None, None], l, theta[None, ...], phi[None, ...])
        harmonics = np.einsum('ij,jmn->imn', rot, sph).real
        Y_rot[i, l, num_degrees-l:num_degrees+l+1] = harmonics

In [25]:
frames = [
    dict(
        name=str(t),
        data=[go.Surface(x=x * np.abs(Y_rot[t, l, num_degrees+m]), y=y * np.abs(Y_rot[t, l, num_degrees+m]),
                         z=z * np.abs(Y_rot[t, l, num_degrees+m]), surfacecolor=Y_rot[t, l, num_degrees+m], 
                         colorscale='RdBu_r')
              for l in range(num_degrees+1)
              for m in range(-l, l+1)],
        traces=list(range((num_degrees+1)**2)),
    )
    for t in range(1, num_timesteps)
]

In [26]:
fig = make_subplots(rows=num_degrees+1, cols=max_order,
                    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x * np.abs(Y_rot[0, l, num_degrees+m]), y=y * np.abs(Y_rot[0, l, num_degrees+m]),
                     z=z * np.abs(Y_rot[0, l, num_degrees+m]), surfacecolor=Y_rot[0, l, num_degrees+m],
                     colorscale='RdBu_r'),
          l+1, num_degrees+m+1
        )
fig.update(frames=frames)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)
fig.write_html("wigner_rot_sph_harmonics.html")
IFrame('./wigner_rot_sph_harmonics.html', 1000, 500)

In [27]:
IFrame('./wigner_rot_sph_harmonics.html', 1000, 500)

In [28]:
frames = [
    dict(
        name=str(t),
        data=[go.Surface(x=x, y=y, z=z, surfacecolor=Y_rot[t, l, num_degrees+m], 
                         colorscale='RdBu_r')
              for l in range(num_degrees+1)
              for m in range(-l, l+1)],
        traces=list(range((num_degrees+1)**2)),
    )
    for t in range(1, num_timesteps)
]

In [29]:
fig = make_subplots(rows=num_degrees+1, cols=max_order,
                    specs=[[{'is_3d': True} for r in range(max_order)] for c in range(num_degrees+1)])

for l in range(num_degrees+1):
    for m in range(-l, l+1):
        fig.add_trace(
          go.Surface(x=x, y=y,
                     z=z, surfacecolor=Y_rot[0, l, num_degrees+m],
                     colorscale='RdBu_r'),
          l+1, num_degrees+m+1
        )
fig.update(frames=frames)

fig.update_layout(layout)
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False)
fig.update_traces(showscale=False)
# fig.show()
fig.write_html("wigner_rot_sph_harmonics.html")
IFrame('./wigner_rot_sph_harmonics.html', 1000, 500)

## Resources

- https://irhum.github.io/blog/spherical-harmonics/
- https://blondegeek.github.io/e3nn_tutorial/data_types.html
- https://docs.e3nn.org/en/latest/api/o3/o3_sh.html
- https://uvagedl.github.io/